In [1]:
# 순서대로 재설치
!pip install numpy
!pip install scipy
!pip install gensim
!pip install nltk
!pip install sentencepiece

In [2]:
import numpy
import pandas
import torch
import nltk
import gensim
import torch
import torch.nn as nn
import pandas as pd
import requests
import re
import sentencepiece as spm
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from gensim.models import KeyedVectors
from io import StringIO

In [3]:
import pandas as pd
import requests
from io import StringIO

# GitHub raw 링크로 직접 다운로드
url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
response = requests.get(url)
df = pd.read_csv(StringIO(response.text))

In [4]:
# 데이터 구조 확인
print("데이터 형태:", df.shape)
print("\n컬럼명:")
print(df.columns.tolist())
print("\n첫 5행:")
print(df.head())

# 결측값 확인
print("\n결측값:")
print(df.isnull().sum())

데이터 형태: (11823, 3)

컬럼명:
['Q', 'A', 'label']

첫 5행:
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0

결측값:
Q        0
A        0
label    0
dtype: int64


In [5]:
# 질문과 답변을 각각 변수에 저장
questions = df['Q'].tolist()  # 또는 df['Q'].values
answers = df['A'].tolist()    # 또는 df['A'].values

# 확인
print(f"질문 개수: {len(questions)}")
print(f"답변 개수: {len(answers)}")

# 첫 몇 개 예시 출력
for i in range(3):
    print(f"\n질문 {i+1}: {questions[i]}")
    print(f"답변 {i+1}: {answers[i]}")

질문 개수: 11823
답변 개수: 11823

질문 1: 12시 땡!
답변 1: 하루가 또 가네요.

질문 2: 1지망 학교 떨어졌어
답변 2: 위로해 드립니다.

질문 3: 3박4일 놀러가고 싶다
답변 3: 여행은 언제나 좋죠.


In [6]:
import re

def preprocess_sentence(sentence):
    # 1. 영문자를 모두 소문자로 변환
    sentence = sentence.lower()
    
    # 2. 영문자, 한글, 숫자, 주요 특수문자만 남기고 나머지 제거
    # 한글: \uAC00-\uD7A3 (가-힣)
    # 영문자: a-z (이미 소문자로 변환됨)
    # 숫자: 0-9
    # 주요 특수문자: . , ? ! ; : - ( ) " ' 등
    sentence = re.sub(r'[^a-z가-힣0-9.,?!;:\-()"\'\s]', '', sentence)
    
    # 3. 연속된 공백을 하나로 정리
    sentence = re.sub(r'\s+', ' ', sentence)
    
    # 4. 앞뒤 공백 제거
    sentence = sentence.strip()
    
    return sentence

# 테스트 함수
def test_preprocess():
    """전처리 함수 테스트"""
    test_sentences = [
        "안녕하세요! 오늘 날씨가 어떤가요?",
        "Hello World! How are you today?",
        "이것은 123개의 숫자와 ABC 영문자가 포함된 문장입니다.",
        "특수문자@#$%^&*를 포함한 문장입니다!!!",
        "여러    공백이    있는    문장입니다.",
        "안녕하세요~ ^^ 좋은 하루 되세요!!! ♥♥♥"
    ]
    
    print("=== 전처리 테스트 ===")
    for i, sentence in enumerate(test_sentences, 1):
        processed = preprocess_sentence(sentence)
        print(f"원본 {i}: {sentence}")
        print(f"처리 {i}: {processed}")
        print()

# 테스트 실행
test_preprocess()

# ChatBot 데이터에 적용하는 예시
def apply_preprocessing_to_data(questions, answers):
    """
    질문과 답변 데이터에 전처리 적용
    
    Args:
        questions (list): 질문 리스트
        answers (list): 답변 리스트
    
    Returns:
        tuple: 전처리된 (질문 리스트, 답변 리스트)
    """
    processed_questions = []
    processed_answers = []
    
    for q, a in zip(questions, answers):
        # 전처리 적용
        processed_q = preprocess_sentence(q)
        processed_a = preprocess_sentence(a)
        
        # 빈 문자열이 아닌 경우만 추가
        if processed_q.strip() and processed_a.strip():
            processed_questions.append(processed_q)
            processed_answers.append(processed_a)
    
    print(f"전처리 전: {len(questions)}개")
    print(f"전처리 후: {len(processed_questions)}개")
    
    return processed_questions, processed_answers

=== 전처리 테스트 ===
원본 1: 안녕하세요! 오늘 날씨가 어떤가요?
처리 1: 안녕하세요! 오늘 날씨가 어떤가요?

원본 2: Hello World! How are you today?
처리 2: hello world! how are you today?

원본 3: 이것은 123개의 숫자와 ABC 영문자가 포함된 문장입니다.
처리 3: 이것은 123개의 숫자와 abc 영문자가 포함된 문장입니다.

원본 4: 특수문자@#$%^&*를 포함한 문장입니다!!!
처리 4: 특수문자를 포함한 문장입니다!!!

원본 5: 여러    공백이    있는    문장입니다.
처리 5: 여러 공백이 있는 문장입니다.

원본 6: 안녕하세요~ ^^ 좋은 하루 되세요!!! ♥♥♥
처리 6: 안녕하세요 좋은 하루 되세요!!!



In [7]:
def preprocess_sentence(sentence):
    """문장 전처리"""
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z가-힣0-9.,?!;:\-()"\'\s]', '', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence.strip()

def build_corpus(source_sentences, target_sentences, tokenize_fn, max_length=40):
    """
    소스 문장과 타겟 문장을 정제하고 토큰화하여 코퍼스를 생성하는 함수
    
    Args:
        source_sentences: 소스 문장 리스트 (예: 질문들)
        target_sentences: 타겟 문장 리스트 (예: 답변들)
        tokenize_fn: 토큰화 함수 (tokenizer.encode_as_ids)
        max_length: 최대 토큰 길이 (기본값: 40)
    
    Returns:
        tuple: (source_corpus, target_corpus) - 토큰화된 코퍼스
    """
    print(f"코퍼스 생성 시작 - 입력 데이터: {len(source_sentences)}개 쌍")
    
    # 1. 전처리 및 토큰화
    processed_sources = []
    processed_targets = []
    source_tokens_list = []
    target_tokens_list = []
    
    for src, tgt in zip(source_sentences, target_sentences):
        # 전처리
        clean_src = preprocess_sentence(src)
        clean_tgt = preprocess_sentence(tgt)
        
        # 토큰화
        src_tokens = tokenize_fn(clean_src)
        tgt_tokens = tokenize_fn(clean_tgt)
        
        # 길이 제한 확인
        if len(src_tokens) <= max_length and len(tgt_tokens) <= max_length:
            processed_sources.append(clean_src)
            processed_targets.append(clean_tgt)
            source_tokens_list.append(src_tokens)
            target_tokens_list.append(tgt_tokens)
    
    print(f"길이 제한 후: {len(processed_sources)}개 쌍")
    
    # 2. 중복 제거 (소스와 타겟 각각 독립적으로, 하지만 쌍은 유지)
    seen_sources = set()
    seen_targets = set()
    final_source_corpus = []
    final_target_corpus = []
    
    for i, (src_text, tgt_text) in enumerate(zip(processed_sources, processed_targets)):
        # 소스나 타겟 중 하나라도 중복이면 해당 쌍 전체 제외
        if src_text not in seen_sources and tgt_text not in seen_targets:
            seen_sources.add(src_text)
            seen_targets.add(tgt_text)
            final_source_corpus.append(source_tokens_list[i])
            final_target_corpus.append(target_tokens_list[i])
    
    print(f"중복 제거 후: {len(final_source_corpus)}개 쌍")
    
    return final_source_corpus, final_target_corpus

def process_chatbot_data(questions, answers):
    """ChatBot 데이터 토큰화 (build_corpus 함수 활용)"""
    print("SentencePiece 토크나이저 훈련 시작")
    
    # 1. 훈련용 텍스트 파일 생성
    with open('chatbot_corpus.txt', 'w', encoding='utf-8') as f:
        for q, a in zip(questions, answers):
            f.write(f"{q}\n")
            f.write(f"{a}\n")
    
    # 2. SentencePiece 모델 훈련
    spm.SentencePieceTrainer.train(
        input='chatbot_corpus.txt',
        model_prefix='chatbot_spm',
        vocab_size=8000,
        model_type='bpe',
        user_defined_symbols='<pad>,<sos>,<eos>'
    )
    
    # 3. 토크나이저 로드
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.load('chatbot_spm.model')
    
    print("토크나이저 훈련 완료")
    print(f"어휘 사전 크기: {tokenizer.get_piece_size()}")
    
    # 4. build_corpus 함수를 사용하여 데이터 토큰화
    que_corpus, ans_corpus = build_corpus(
        source_sentences=questions,
        target_sentences=answers,
        tokenize_fn=tokenizer.encode_as_ids,
        max_length=40
    )
    
    print(f"최종 토큰화 완료: {len(que_corpus)}개 문장 쌍")
    
    # 5. 샘플 확인
    print("\n토큰화 결과 샘플:")
    for i in range(min(3, len(que_corpus))):
        print(f"질문 토큰: {que_corpus[i][:10]}...")
        print(f"답변 토큰: {ans_corpus[i][:10]}...")
        print(f"복원 질문: {tokenizer.decode_ids(que_corpus[i])}")
        print(f"복원 답변: {tokenizer.decode_ids(ans_corpus[i])}")
        print("-" * 50)
    
    return que_corpus, ans_corpus, tokenizer

# ==============================================
# 실행 부분
# ==============================================

# 먼저 데이터가 있는지 확인
if 'questions' in globals() and 'answers' in globals():
    print("데이터 발견! 토큰화 시작...")
    que_corpus, ans_corpus, tokenizer = process_chatbot_data(questions, answers)
    
    # 특수 토큰 ID 확인
    PAD_IDX = tokenizer.piece_to_id('<pad>')
    SOS_IDX = tokenizer.piece_to_id('<sos>')
    EOS_IDX = tokenizer.piece_to_id('<eos>')
    
    print(f"\n특수 토큰 ID:")
    print(f"PAD: {PAD_IDX}, SOS: {SOS_IDX}, EOS: {EOS_IDX}")
    
else:
    print("❌ questions, answers 변수가 없습니다!")
    print("💡 먼저 Step 1(데이터 로드)를 실행해주세요:")

데이터 발견! 토큰화 시작...
SentencePiece 토크나이저 훈련 시작
토크나이저 훈련 완료
어휘 사전 크기: 8000
코퍼스 생성 시작 - 입력 데이터: 11823개 쌍


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: chatbot_corpus.txt
  input_format: 
  model_prefix: chatbot_spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <pad>
  user_defined_symbols: <sos>
  user_defined_symbols: <eos>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
 

길이 제한 후: 11823개 쌍
중복 제거 후: 7737개 쌍
최종 토큰화 완료: 7737개 문장 쌍

토큰화 결과 샘플:
질문 토큰: [5568, 6957, 3209, 7063]...
답변 토큰: [4491, 213, 5938, 6916]...
복원 질문: 12시 땡!
복원 답변: 하루가 또 가네요.
--------------------------------------------------
질문 토큰: [346, 6924, 7230, 1008, 2460]...
답변 토큰: [1621, 6424, 6916]...
복원 질문: 1지망 학교 떨어졌어
복원 답변: 위로해 드립니다.
--------------------------------------------------
질문 토큰: [472, 7425, 7392, 6965, 3502, 200]...
답변 토큰: [5138, 1359, 381, 6916]...
복원 질문: 3박4일 놀러가고 싶다
복원 답변: 여행은 언제나 좋죠.
--------------------------------------------------

특수 토큰 ID:
PAD: 3, SOS: 4, EOS: 5


In [8]:
from sklearn.model_selection import train_test_split

def split_data(que_corpus, ans_corpus, test_size=0.2, random_state=42):
    """토큰화된 데이터를 train/test로 분할"""
    
    train_que, test_que, train_ans, test_ans = train_test_split(
        que_corpus, ans_corpus, 
        test_size=test_size, 
        random_state=random_state
    )
    
    print(f"Train: {len(train_que)}개, Test: {len(test_que)}개")
    
    return train_que, test_que, train_ans, test_ans

# 실행
if 'que_corpus' in globals() and 'ans_corpus' in globals():
    train_que, test_que, train_ans, test_ans = split_data(que_corpus, ans_corpus)
else:
    print("que_corpus, ans_corpus 변수가 없습니다!")

Train: 6189개, Test: 1548개


In [11]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="chatbot",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang
    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')
    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=%s --model_prefix=%s --vocab_size=%d --pad_id=%d --bos_id=%d --eos_id=%d --unk_id=%d'
        % (file, model, vocab_size, pad_id, bos_id, eos_id, unk_id)
    )
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)
    return tokenizer

print("슝=3")

슝=3


In [13]:
VOCAB_SIZE = 8000
corpus = questions + answers  # 리스트 합치기
tokenizer = generate_tokenizer(corpus, VOCAB_SIZE, 'chatbot')
tokenizer.set_encode_extra_options("bos:eos")

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=./chatbot_corpus.txt --model_prefix=chatbot_spm --vocab_size=8000 --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./chatbot_corpus.txt
  input_format: 
  model_prefix: chatbot_spm
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_voca

True

In [14]:
def make_corpus(sentences, tokenizer):
    corpus = []
    for sentence in tqdm(sentences):
        tokens = tokenizer.encode_as_ids(sentence)
        corpus.append(tokens)
    return corpus

print('슝=3')

슝=3


In [16]:
from tqdm import tqdm

In [17]:
questions_corpus = make_corpus(questions, tokenizer)
answers_corpus = make_corpus(answers, tokenizer)

100%|██████████| 11823/11823 [00:00<00:00, 158634.86it/s]


In [18]:
from sklearn.model_selection import train_test_split

train_que, test_que, train_ans, test_ans = train_test_split(
    questions_corpus, answers_corpus, 
    test_size=0.2, 
    random_state=42
)

print(f"Train: {len(train_que)}개, Test: {len(test_que)}개")

Train: 9458개, Test: 2365개


In [19]:
MAX_LEN = 50
def pad_sequences_custom(sequences, max_len=50, pad_value=0):
   """
   sequences: list of list (각 문장별 토큰 ID 리스트)
   max_len: 고정할 최대 시퀀스 길이
   pad_value: 패딩에 사용할 값 (일반적으로 0)
   """
   padded_sequences = []
   for seq in sequences:
       # 초과 길이는 자르고
       if len(seq) > max_len:
           seq = seq[:max_len]
       # 부족한 길이는 pad_value로 채우기
       else:
           seq = seq + [pad_value] * (max_len - len(seq))
       padded_sequences.append(seq)
   # 최종적으로 torch.Tensor로 변환 (shape: [batch_size, max_len])
   return torch.tensor(padded_sequences, dtype=torch.long)
enc_ndarray = pad_sequences_custom(train_que, max_len=MAX_LEN, pad_value=0)
dec_ndarray = pad_sequences_custom(train_ans, max_len=MAX_LEN, pad_value=0)
print(enc_ndarray.shape)  # 예) [batch_size, 50]
print(dec_ndarray.shape)  # 예) [batch_size, 50]
print("슝=3")

torch.Size([9458, 50])
torch.Size([9458, 50])
슝=3


In [20]:
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = TensorDataset(enc_ndarray, dec_ndarray)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

print("슝=3")

슝=3


In [21]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table
print("슝=3")

슝=3


In [22]:
def generate_padding_mask(seq: torch.Tensor) -> torch.Tensor:
    """
    seq: shape [batch_size, seq_len]의 입력 (토큰 ID 텐서)
    반환: shape [batch_size, 1, 1, seq_len]의 패딩 마스크
         (seq == 0)인 위치가 1, 나머지는 0
    """
    # (seq == 0)은 불리언 텐서를 반환 -> float()로 형변환 -> (1.0 or 0.0)
    # 차원 확장: [batch_size, seq_len] → [batch_size, 1, 1, seq_len]
    return (seq == 0).unsqueeze(1).unsqueeze(2).float()


def generate_lookahead_mask(size: int) -> torch.Tensor:
    """
    size: 문장(시퀀스) 길이
    반환: shape [size, size],
         i < j (대각선 위)에 해당하는 위치가 1, 아닌 곳은 0
         (미래 토큰을 가리기 위한 마스크)
    """
    # triu(diagonal=1)은 주대각선 위가 1, 아래가 0인 텐서를 만들어 줌
    return torch.triu(torch.ones(size, size), diagonal=1)


def generate_masks(src: torch.Tensor, tgt: torch.Tensor):
    """
    src, tgt: shape [batch_size, seq_len]
    3가지 마스크를 반환:
      - enc_mask: 인코더 입력용 패딩 마스크
      - dec_enc_mask: 디코더-인코더 어텐션용 패딩 마스크
      - dec_mask: 디코더 자기어텐션용 마스크(룩어헤드 + 패딩)

    각각의 shape:
      - enc_mask, dec_enc_mask: [batch_size, 1, 1, src_seq_len]
      - dec_mask: [batch_size, 1, tgt_seq_len, tgt_seq_len]
    """
    # 1) 인코더 입력용 패딩 마스크
    enc_mask = generate_padding_mask(src)
    # 2) 디코더에서 인코더 값을 볼 때 사용하는 마스크 (src 마스크 재사용)
    dec_enc_mask = generate_padding_mask(src)

    # 3) 디코더 자기어텐션 마스크 (미래 토큰 방지 룩어헤드 + tgt 자체 패딩 마스크)
    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])  # [tgt_seq_len, tgt_seq_len]
    dec_tgt_padding_mask = generate_padding_mask(tgt)           # [batch_size, 1, 1, tgt_seq_len]

    # 룩어헤드 마스크를 (batch 차원과 head 차원을 가상으로) 확장
    dec_lookahead_mask = dec_lookahead_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, seq_len, seq_len]

    # 패딩 + 룩어헤드 마스크 병합
    # 브로드캐스팅에 의해 shape [batch_size, 1, tgt_seq_len, tgt_seq_len]이 됨

    dec_tgt_padding_mask = dec_tgt_padding_mask.to(device)
    dec_lookahead_mask = dec_lookahead_mask.to(device)

    dec_mask = torch.max(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

print("슝=3")

슝=3


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        # d_model을 num_heads로 나눈 만큼이 각 head가 담당할 차원 수
        self.depth = d_model // num_heads

        # Query, Key, Value를 구하는 선형 레이어
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        # 최종적으로 head들의 출력을 결합해주는 선형 레이어
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        Q, K, V:  [batch_size, num_heads, seq_len, depth]
        mask:     [batch_size, 1, seq_len, seq_len] 혹은
                  [batch_size, num_heads, seq_len, seq_len]
                  (어텐션에서 제외할 위치=1, 사용할 위치=0)
        """
        # d_k = depth
        d_k = Q.size(-1)  # K.shape[-1]도 동일
        # Q와 K의 전치 곱: (batch_size, num_heads, seq_len, seq_len)
        QK = torch.matmul(Q, K.transpose(-1, -2))

        # 스케일링
        scaled_qk = QK / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

        # 마스크가 있는 경우 -1e9(매우 작은 수)를 더하여 softmax 후 확률이 0에 가깝도록 처리
        if mask is not None:
            scaled_qk = scaled_qk + (mask * -1e9)

        attentions = F.softmax(scaled_qk, dim=-1)  # (batch_size, num_heads, seq_len, seq_len)
        out = torch.matmul(attentions, V)         # (batch_size, num_heads, seq_len, depth)

        return out, attentions

    def split_heads(self, x):
        """
        x: [batch_size, seq_len, d_model]
        반환: [batch_size, num_heads, seq_len, depth]
        """
        bsz, seq_len, _ = x.size()
        # d_model -> (num_heads * depth)이므로 view로 재배치
        x = x.view(bsz, seq_len, self.num_heads, self.depth)
        # (batch_size, seq_len, num_heads, depth) -> (batch_size, num_heads, seq_len, depth)
        x = x.permute(0, 2, 1, 3)
        return x

    def combine_heads(self, x):
        """
        x: [batch_size, num_heads, seq_len, depth]
        반환: [batch_size, seq_len, d_model]
        """
        bsz, num_heads, seq_len, depth = x.size()
        # (batch_size, num_heads, seq_len, depth) -> (batch_size, seq_len, num_heads, depth)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(bsz, seq_len, self.d_model)
        return x

    def forward(self, Q, K, V, mask=None):
        """
        Q, K, V: [batch_size, seq_len, d_model]
        mask:    [batch_size, 1, seq_len, seq_len] 혹은
                 [batch_size, num_heads, seq_len, seq_len]
        """
        # W_q, W_k, W_v는 각각 (d_model -> d_model) 선형 변환
        WQ = self.W_q(Q)  # [batch_size, seq_len, d_model]
        WK = self.W_k(K)  # [batch_size, seq_len, d_model]
        WV = self.W_v(V)  # [batch_size, seq_len, d_model]

        # 멀티헤드 분할
        WQ_splits = self.split_heads(WQ)  # [batch_size, num_heads, seq_len, depth]
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        # Scaled dot-product attention
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask
        )

        # head 결과 결합 후 최종 선형
        out = self.combine_heads(out)  # [batch_size, seq_len, d_model]
        out = self.linear(out)         # [batch_size, seq_len, d_model]

        return out, attention_weights

print("슝=3")

슝=3


In [24]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.fc1(x))  # 첫 번째 Dense + ReLU
        out = self.fc2(out)          # 두 번째 Dense
        return out

print("슝=3")

슝=3


In [25]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        # nn.LayerNorm은 마지막 차원(d_model)을 기준으로 정규화
        self.norm_1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm_2 = nn.LayerNorm(d_model, eps=1e-6)

        self.do = nn.Dropout(dropout)

    def forward(self, x, mask):
        # Multi-Head Attention 단계
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out = out + residual  # residual connection

        # Position-Wise Feed Forward 단계
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out = out + residual  # residual connection

        return out, enc_attn

print("슝=3")

슝=3


In [26]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm_2 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm_3 = nn.LayerNorm(d_model, eps=1e-6)

        self.do = nn.Dropout(dropout)

    def forward(self, x, enc_out, dec_enc_mask, padding_mask):
        # Masked Multi-Head Attention
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, mask=padding_mask)
        out = self.do(out)
        out = out + residual

        # Encoder-Decoder Multi-Head Attention (주의: Q, K, V 순서)
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out = out + residual

        # Position-Wise Feed Forward Network
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out = out + residual

        return out, dec_attn, dec_enc_attn

print("슝=3")

슝=3


In [27]:
class Encoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = nn.ModuleList(
            [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )
        self.do = nn.Dropout(dropout)  # 필요 시 입력에 dropout 적용 가능

    def forward(self, x, mask):
        out = x
        enc_attns = []
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        return out, enc_attns

# 사용 예시: Encoder 인스턴스 생성 후 forward 호출
# encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
# out, enc_attns = encoder(x, mask)
print("슝=3")

슝=3


In [28]:
class Decoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = nn.ModuleList(
            [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )

    def forward(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
        dec_attns = []
        dec_enc_attns = []
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
        return out, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [29]:
import math

class Transformer(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff,
                 src_vocab_size, tgt_vocab_size, pos_len,
                 dropout=0.2, shared_fc=True, shared_emb=False):
        super(Transformer, self).__init__()
        # d_model은 스케일링에 사용되므로 float으로 저장
        self.d_model = float(d_model)

        # Embedding 레이어: shared_emb True면 동일한 임베딩을 사용합니다.
        if shared_emb:
            self.enc_emb = self.dec_emb = nn.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = nn.Embedding(src_vocab_size, d_model)
            self.dec_emb = nn.Embedding(tgt_vocab_size, d_model)

        # Positional encoding (넘파이 버전 결과를 torch.Tensor로 변환)
        pos_encoding_np = positional_encoding(pos_len, d_model)
        # 파라미터로 등록하지 않고 고정값이므로 buffer로 등록합니다.
        self.register_buffer("pos_encoding", torch.tensor(pos_encoding_np, dtype=torch.float32))

        self.do = nn.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = nn.Linear(d_model, tgt_vocab_size)

        self.shared_fc = shared_fc
        if shared_fc:
            # fc 레이어와 디코더 임베딩의 weight를 공유합니다.
            self.fc.weight = self.dec_emb.weight

    def embedding(self, emb, x):
        """
        emb: 임베딩 레이어
        x: [batch_size, seq_len] (토큰 인덱스)
        """
        seq_len = x.size(1)
        out = emb(x)  # [batch_size, seq_len, d_model]
        if self.shared_fc:
            out = out * math.sqrt(self.d_model)
        # pos_encoding: [pos_len, d_model] → [1, pos_len, d_model] 후 슬라이싱
        out = out + self.pos_encoding[:seq_len, :].unsqueeze(0)
        out = self.do(out)
        return out

    def forward(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        """
        enc_in: [batch_size, src_seq_len]
        dec_in: [batch_size, tgt_seq_len]
        enc_mask, dec_enc_mask, dec_mask: 마스킹 텐서들
        """
        # Embedding 및 positional encoding 적용
        enc_in_emb = self.embedding(self.enc_emb, enc_in)
        dec_in_emb = self.embedding(self.dec_emb, dec_in)

        # Encoder와 Decoder 통과
        enc_out, enc_attns = self.encoder(enc_in_emb, enc_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in_emb, enc_out, dec_enc_mask, dec_mask)

        logits = self.fc(dec_out)
        return logits, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [31]:
import numpy as np

In [32]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

transformer = transformer.to(device)

d_model = 512

print("슝=3")

슝=3


In [33]:
class LearningRateScheduler:
    def __init__(self, d_model, warmup_steps=60): # 4000
        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        # step을 float으로 변환하여 지수 연산이 제대로 수행되도록 함
        step = float(step)
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        return (self.d_model ** -0.5) * min(arg1, arg2)

print("슝=3")

슝=3


In [34]:
# Learning Rate 인스턴스 선언
learning_rate = LearningRateScheduler(d_model)

# 초기 lr은 스텝 1에 해당하는 값으로 설정합니다.
optimizer = torch.optim.Adam(transformer.parameters(),
                             lr=learning_rate(1),
                             betas=(0.9, 0.98),
                             eps=1e-9)

print("슝=3")

슝=3


In [35]:
def loss_function(real, pred):
    """
    real: [batch_size, seq_len] (정답 토큰 인덱스)
    pred: [batch_size, seq_len, num_classes] (모델의 raw logits)
    """

    real = real.to(device)
    pred = pred.to(device)

    # 예측 값을 (N, C) 형태로 flatten하고, 정답도 flatten하여 개별 손실 값을 구함
    loss_ = F.cross_entropy(pred.contiguous().view(-1, pred.size(-1)), real.contiguous().view(-1), reduction='none')
    # 다시 (batch_size, seq_len)로 reshape
    loss_ = loss_.view(real.size())

    # real이 0이 아닌 위치에 대한 마스크 생성 (0이면 패딩 토큰)
    mask = (real != 0).float()
    loss_ = loss_ * mask

    # 전체 손실 합을 마스크 합으로 나누어 평균 손실 계산
    return loss_.sum() / mask.sum()

print("슝=3")

슝=3


In [36]:
def train_step(src, tgt, model, optimizer):
    model.train()  # 모델을 training 모드로 전환
    optimizer.zero_grad()

    # tgt의 오른쪽 시프트: decoder input과 gold target 분리
    tgt_in = tgt[:, :-1]  # Decoder의 입력
    gold = tgt[:, 1:]     # Decoder의 정답(target)

    # 마스크 생성 (generate_masks는 PyTorch용으로 변환된 함수여야 합니다)
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    src = src.to(device)
    tgt_in = tgt_in.to(device)
    enc_mask = enc_mask.to(device)
    dec_enc_mask = dec_enc_mask.to(device)
    dec_mask = dec_mask.to(device)

    # 모델 forward pass
    predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)

    # loss 계산
    loss = loss_function(gold, predictions)

    # 역전파 수행 및 파라미터 업데이트
    loss.backward()
    optimizer.step()

    return loss, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [37]:
%%time

EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0.0
    dataset_count = len(train_dataloader)  # train_loader는 PyTorch DataLoader입니다.
    tqdm_bar = tqdm(total=dataset_count)

    for batch, (src, tgt) in enumerate(train_dataloader):
        # train_step 함수는 (loss, enc_attns, dec_attns, dec_enc_attns)를 반환합니다.
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)

        total_loss += loss.item()  # PyTorch에서는 loss.numpy() 대신 loss.item() 사용
        tqdm_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
        tqdm_bar.update(1)

    tqdm_bar.close()
    print(f"Epoch {epoch+1}, Loss: {total_loss / dataset_count:.4f}")

100%|██████████| 148/148 [00:19<00:00,  7.47it/s, Batch Loss=6258.9404]


Epoch 1, Loss: 7705.0352


100%|██████████| 148/148 [00:17<00:00,  8.66it/s, Batch Loss=5058.3076]


Epoch 2, Loss: 5343.2152


100%|██████████| 148/148 [00:16<00:00,  9.02it/s, Batch Loss=4518.4399]

Epoch 3, Loss: 4749.6545
CPU times: user 59.5 s, sys: 408 ms, total: 60 s
Wall time: 53.3 s


In [38]:
import random
from gensim.models import KeyedVectors
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z가-힣0-9.,?!;:\-()"\'\s]', '', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence.strip()

def enhanced_lexical_sub(sentence, wv, num_substitutions=1):
    tokens = sentence.split()
    valid_tokens = [tok for tok in tokens if tok in wv]
    
    if not valid_tokens:
        return sentence
    
    max_subs = min(num_substitutions, len(valid_tokens), max(1, len(tokens) // 3))
    selected_tokens = random.sample(valid_tokens, max_subs)
    
    new_tokens = tokens.copy()
    for selected_tok in selected_tokens:
        try:
            similar_words = wv.most_similar(selected_tok, topk=3)
            similar_word = random.choice(similar_words)[0]
            new_tokens = [similar_word if tok == selected_tok else tok for tok in new_tokens]
        except:
            continue
    
    return " ".join(new_tokens)

def augmented_corpus(questions, answers, tokenizer, wv_model, max_length=40):
    print(f"원본 데이터: {len(questions)}개")
    
    final_questions = []
    final_answers = []
    
    for q, a in zip(questions, answers):
        clean_q = preprocess_sentence(q)
        clean_a = preprocess_sentence(a)
        
        try:
            # 원본
            final_questions.append(clean_q)
            final_answers.append(clean_a)
            
            # 증강 질문 + 원본 답변
            aug_question = enhanced_lexical_sub(clean_q, wv_model)
            final_questions.append(aug_question)
            final_answers.append(clean_a)
            
            # 원본 질문 + 증강 답변
            aug_answer = enhanced_lexical_sub(clean_a, wv_model)
            final_questions.append(clean_q)
            final_answers.append(aug_answer)
                
        except:
            final_questions.extend([clean_q] * 3)
            final_answers.extend([clean_a] * 3)
    
    # 토큰화
    que_corpus = []
    ans_corpus = []
    
    for q, a in zip(final_questions, final_answers):
        q_tokens = tokenizer.encode_as_ids(q)
        a_tokens = tokenizer.encode_as_ids(a)
        
        if len(q_tokens) <= max_length and len(a_tokens) <= max_length:
            que_corpus.append(q_tokens)
            ans_corpus.append(a_tokens)
    
    print(f"증강 완료: {len(que_corpus)}개")
    return que_corpus, ans_corpus

def load_korean_word_vectors(model_path='ko.vec'):
    try:
        wv = KeyedVectors.load_word2vec_format(model_path, binary=False, limit=100000)
        print(f"워드벡터 로드 성공: {len(wv.key_to_index):,}개 단어")
        return wv
    except:
        print("워드벡터 로드 실패")
        return None

# 실행
wv_model = load_korean_word_vectors('ko.vec')
if wv_model:
    que_corpus, ans_corpus = augmented_corpus(questions, answers, tokenizer, wv_model)
    
    # 결과 확인
    print(f"증강 전: {len(questions)}개")
    print(f"증강 후: {len(que_corpus)}개")
    print(f"증강 비율: {len(que_corpus)/len(questions):.1f}배")
    print("슝=3")
else:
    print("워드벡터 모델이 없어 증강을 건너뜁니다")

워드벡터 로드 성공: 30,186개 단어
원본 데이터: 11823개
증강 완료: 35469개
증강 전: 11823개
증강 후: 35469개
증강 비율: 3.0배
슝=3


In [39]:
# 특수 토큰 ID 확인
START_IDX = tokenizer.piece_to_id('<s>')  # 또는 '<sos>'
END_IDX = tokenizer.piece_to_id('</s>')   # 또는 '<eos>'

print(f"START 토큰 ID: {START_IDX}")
print(f"END 토큰 ID: {END_IDX}")

# ans_corpus에 시작/끝 토큰 추가
ans_corpus_with_tokens = []
for ans_seq in ans_corpus:
    # [START] + 원본 시퀀스 + [END]
    new_seq = [START_IDX] + ans_seq + [END_IDX]
    ans_corpus_with_tokens.append(new_seq)

ans_corpus = ans_corpus_with_tokens
print(f"토큰 추가 완료: {len(ans_corpus)}개 시퀀스")
print("슝=3")

START 토큰 ID: 1
END 토큰 ID: 2
토큰 추가 완료: 35469개 시퀀스
슝=3


In [40]:
# 전체 데이터에서 고유 토큰 추출하여 단어 사전 구축
all_tokens = set()

# que_corpus와 ans_corpus의 모든 토큰 수집
for seq in que_corpus:
   all_tokens.update(seq)
for seq in ans_corpus:
   all_tokens.update(seq)

# 단어 사전 구축 (토큰 ID -> 인덱스 매핑)
vocab_size = len(all_tokens)
token_to_idx = {token: idx for idx, token in enumerate(sorted(all_tokens))}
idx_to_token = {idx: token for token, idx in token_to_idx.items()}

print(f"전체 어휘 사전 크기: {vocab_size}")

# 토큰 ID를 인덱스로 변환하여 벡터화
def vectorize_sequences(sequences, token_to_idx):
   vectorized = []
   for seq in sequences:
       vec_seq = [token_to_idx[token] for token in seq if token in token_to_idx]
       vectorized.append(vec_seq)
   return vectorized

# 벡터화 실행
enc_train = vectorize_sequences(que_corpus, token_to_idx)
dec_train = vectorize_sequences(ans_corpus, token_to_idx)

print(f"enc_train: {len(enc_train)}개 시퀀스")
print(f"dec_train: {len(dec_train)}개 시퀀스")
print("슝=3")

전체 어휘 사전 크기: 7857
enc_train: 35469개 시퀀스
dec_train: 35469개 시퀀스
슝=3


In [43]:
%%time

EPOCHS = 15

for epoch in range(EPOCHS):
    total_loss = 0.0
    dataset_count = len(train_dataloader)  # train_loader는 PyTorch DataLoader입니다.
    tqdm_bar = tqdm(total=dataset_count)

    for batch, (src, tgt) in enumerate(train_dataloader):
        # train_step 함수는 (loss, enc_attns, dec_attns, dec_enc_attns)를 반환합니다.
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)

        total_loss += loss.item()  # PyTorch에서는 loss.numpy() 대신 loss.item() 사용
        tqdm_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
        tqdm_bar.update(1)

    tqdm_bar.close()
    print(f"Epoch {epoch+1}, Loss: {total_loss / dataset_count:.4f}")

100%|██████████| 148/148 [00:17<00:00,  8.44it/s, Batch Loss=4096.9268]


Epoch 1, Loss: 3976.8963


100%|██████████| 148/148 [00:17<00:00,  8.65it/s, Batch Loss=3866.5576]


Epoch 2, Loss: 3853.3530


100%|██████████| 148/148 [00:16<00:00,  8.98it/s, Batch Loss=3674.2617]


...

Epoch 10, Loss: 3149.8557


100%|██████████| 148/148 [00:16<00:00,  8.92it/s, Batch Loss=3181.5254]


Epoch 11, Loss: 3087.9570


 26%|██▋       | 39/148 [00:04<00:12,  8.83it/s, Batch Loss=3049.3149]

KeyboardInterrupt: 

In [46]:
# 현재 모델 상태 저장
torch.save({
    'epoch': 8,
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses if 'train_losses' in globals() else [],
}, 'transformer_epoch8_backup.pt')

print("8에포크 모델 저장 완료!")

8에포크 모델 저장 완료!


In [49]:
# 하이퍼파라미터 수정된 새 모델
transformer_new = Transformer(
    n_layers=1,           # 2 → 1로 감소
    d_model=256,          # 512 → 256으로 감소
    n_heads=4,            # 8 → 4로 감소
    d_ff=1024,            # 2048 → 1024로 감소
    src_vocab_size=VOCAB_SIZE,  # 빠진 인자
    tgt_vocab_size=VOCAB_SIZE,  # 빠진 인자
    pos_len=200,          # 빠진 인자 (기존과 동일)
    dropout=0.5,          # 0.3 → 0.5로 증가
    shared_fc=True,       # 기존과 동일
    shared_emb=True       # 기존과 동일
).to(device)

# 새 옵티마이저
optimizer_new = torch.optim.Adam(transformer_new.parameters(), lr=0.0005)

print("새 모델 생성 완료!")
print(f"파라미터 수: {sum(p.numel() for p in transformer_new.parameters()):,}")

새 모델 생성 완료!
파라미터 수: 3,899,200


In [50]:
%%time

EPOCHS = 15

for epoch in range(EPOCHS):
    total_loss = 0.0
    dataset_count = len(train_dataloader)  # train_loader는 PyTorch DataLoader입니다.
    tqdm_bar = tqdm(total=dataset_count)

    for batch, (src, tgt) in enumerate(train_dataloader):
        # train_step 함수는 (loss, enc_attns, dec_attns, dec_enc_attns)를 반환합니다.
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)

        total_loss += loss.item()  # PyTorch에서는 loss.numpy() 대신 loss.item() 사용
        tqdm_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
        tqdm_bar.update(1)

    tqdm_bar.close()
    print(f"Epoch {epoch+1}, Loss: {total_loss / dataset_count:.4f}")


 26%|██▋       | 39/148 [04:55<13:46,  7.58s/it, Batch Loss=3049.3149]

100%|██████████| 148/148 [00:19<00:00,  7.78it/s, Batch Loss=2915.2217]


Epoch 1, Loss: 3016.4459


100%|██████████| 148/148 [00:17<00:00,  8.62it/s, Batch Loss=2813.2981]


Epoch 2, Loss: 2953.6212


100%|██████████| 148/148 [00:16<00:00,  8.98it/s, Batch Loss=2807.0603]


...

Epoch 13, Loss: 2392.9724


100%|██████████| 148/148 [00:16<00:00,  8.86it/s, Batch Loss=2180.3779]


Epoch 14, Loss: 2345.9440


100%|██████████| 148/148 [00:16<00:00,  8.88it/s, Batch Loss=2110.0759]

Epoch 15, Loss: 2303.0242
CPU times: user 4min 44s, sys: 1.44 s, total: 4min 46s
Wall time: 4min 12s


In [53]:
# 현재 모델 상태 저장
torch.save({
    'epoch': 15,
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': 2303.0242
}, 'transformer_epoch15.pt')

print("15에포크 모델 저장 완료!")

15에포크 모델 저장 완료!


In [51]:
# 배치 사이즈 수정
BATCH_SIZE = 128

# 기존 DataLoader 업데이트
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# 새 모델 (기존 이름 그대로)
transformer = Transformer(
    n_layers=NUM_ENCODER_LAYERS,  # 3
    d_model=D_MODEL,              # 256
    n_heads=N_HEAD,               # 8
    d_ff=D_FF,                    # 512
    src_vocab_size=VOCAB_SIZE,    
    tgt_vocab_size=VOCAB_SIZE,    
    pos_len=200,                  
    dropout=DROPOUT,              # 0.2
    shared_fc=True,               
    shared_emb=True               
).to(device)

# 기존 옵티마이저 업데이트
optimizer = torch.optim.Adam(transformer.parameters(), lr=LEARNING_RATE)

print("설정 업데이트 완료!")

NameError: name 'NUM_ENCODER_LAYERS' is not defined

In [54]:
# 모델 생성
transformer = Transformer(
   n_layers=3,
   d_model=256,
   n_heads=8,
   d_ff=512,
   src_vocab_size=VOCAB_SIZE,
   tgt_vocab_size=VOCAB_SIZE,
   pos_len=200,
   dropout=0.2,
   shared_fc=True,
   shared_emb=True
).to(device)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001)
BATCH_SIZE = 128
EPOCHS = 50

In [61]:
def train_epoch(model, dataloader, optimizer, criterion, device, epoch):
   model.train()
   epoch_loss = 0
   
   for batch_idx, (src, tgt) in enumerate(dataloader):
       src, tgt = src.to(device), tgt.to(device)
       
       tgt_input = tgt[:, :-1]
       tgt_output = tgt[:, 1:]
       
       enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_input)
       
       optimizer.zero_grad()
       
       model_output = model(src, tgt_input, enc_mask, dec_enc_mask, dec_mask)
       
       if isinstance(model_output, tuple):
           output = model_output[0]
       else:
           output = model_output
       
       output = output.reshape(-1, output.shape[-1])
       tgt_output = tgt_output.reshape(-1)
       
       loss = criterion(output, tgt_output)
       loss.backward()
       optimizer.step()
       
       epoch_loss += loss.item()
       
       if batch_idx % 100 == 0:
           print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
   
   return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device, epoch):
   model.eval()
   epoch_loss = 0
   
   with torch.no_grad():
       for src, tgt in dataloader:
           src, tgt = src.to(device), tgt.to(device)
           
           tgt_input = tgt[:, :-1]
           tgt_output = tgt[:, 1:]
           
           enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_input)
           
           model_output = model(src, tgt_input, enc_mask, dec_enc_mask, dec_mask)
           
           if isinstance(model_output, tuple):
               output = model_output[0]
           else:
               output = model_output
           
           output = output.reshape(-1, output.shape[-1])
           tgt_output = tgt_output.reshape(-1)
           
           loss = criterion(output, tgt_output)
           epoch_loss += loss.item()
   
   return epoch_loss / len(dataloader)

# 훈련 시작
print("\n훈련 시작")
best_val_loss = float('inf')
early_stop_counter = 0
patience = 3
model_save_path = 'best_transformer.pt'
train_losses, val_losses = [], []

for epoch in range(1, EPOCHS + 1):
   start_time = time.time()
   
   train_loss = train_epoch(transformer, train_dataloader, optimizer, criterion, device, epoch)
   val_loss = evaluate(transformer, val_dataloader, criterion, device, epoch)
   
   end_time = time.time()
   epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
   
   train_losses.append(train_loss)
   val_losses.append(val_loss)
   
   print(f'Epoch: {epoch:02} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
   print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}')
   
   if val_loss < best_val_loss:
       best_val_loss = val_loss
       torch.save(transformer.state_dict(), model_save_path)
       early_stop_counter = 0
       print("\t-> 검증 손실 감소, 모델 저장 완료.")
   else:
       early_stop_counter += 1
       print(f"\t-> 검증 손실 증가. ({early_stop_counter}/{patience})")
       
       if early_stop_counter >= patience:
           print("Early stopping. 훈련을 중단합니다.")
           break

print("\n훈련 완료!")
print("슝=3")


훈련 시작
Epoch 1, Batch 0, Loss: 4134.5352
Epoch: 01 | Time: 0m 8s
	Train Loss: 3154.857 | Val. Loss: 2391.747
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 2, Batch 0, Loss: 2611.2571
Epoch: 02 | Time: 0m 11s
	Train Loss: 2181.410 | Val. Loss: 1601.866
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 3, Batch 0, Loss: 1807.4886
Epoch: 03 | Time: 0m 8s
	Train Loss: 1710.566 | Val. Loss: 1357.760
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 4, Batch 0, Loss: 1612.2356
Epoch: 04 | Time: 0m 7s
	Train Loss: 1558.879 | Val. Loss: 1230.790
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 5, Batch 0, Loss: 1468.7509
Epoch: 05 | Time: 0m 7s
	Train Loss: 1465.160 | Val. Loss: 1153.562
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 6, Batch 0, Loss: 1415.3254
Epoch: 06 | Time: 0m 7s
	Train Loss: 1397.196 | Val. Loss: 1103.059
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 7, Batch 0, Loss: 1381.6857
Epoch: 07 | Time: 0m 7s
	Train Loss: 1348.144 | Val. Loss: 1055.165
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 8, Batch 0, Loss: 1342.2532
Epoch: 08 | Time: 0m 7s
	Train Loss: 1297.734 | Val. Loss: 1011.351
	-> 검증 손

In [62]:
# 현재 모델 상태 저장 (백업용)
torch.save({
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses
}, 'transformer_loss300_backup.pt')

print("현재 모델 백업 완료!")

# 추가 훈련 (더 많은 에포크)
EPOCHS = 30  # 추가로 30 에포크 더
print(f"\n추가 훈련 시작 ({EPOCHS} 에포크)")

for epoch in range(1, EPOCHS + 1):
    start_time = time.time()
    
    train_loss = train_epoch(transformer, train_dataloader, optimizer, criterion, device, epoch)
    val_loss = evaluate(transformer, val_dataloader, criterion, device, epoch)
    
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f'Epoch: {epoch:02} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(transformer.state_dict(), model_save_path)
        early_stop_counter = 0
        print("\t-> 검증 손실 감소, 모델 저장 완료.")
    else:
        early_stop_counter += 1
        print(f"\t-> 검증 손실 증가. ({early_stop_counter}/{patience})")
        
        if early_stop_counter >= patience:
            print("Early stopping. 훈련을 중단합니다.")
            break

print("\n추가 훈련 완료!")
print("슝=3")

현재 모델 백업 완료!

추가 훈련 시작 (30 에포크)
Epoch 1, Batch 0, Loss: 352.3074
Epoch: 01 | Time: 0m 7s
	Train Loss: 348.598 | Val. Loss: 242.862
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 2, Batch 0, Loss: 356.8356
Epoch: 02 | Time: 0m 7s
	Train Loss: 337.344 | Val. Loss: 234.976
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 3, Batch 0, Loss: 341.1765
Epoch: 03 | Time: 0m 8s
	Train Loss: 326.085 | Val. Loss: 229.625
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 4, Batch 0, Loss: 339.4576
Epoch: 04 | Time: 0m 8s
	Train Loss: 315.818 | Val. Loss: 221.009
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 5, Batch 0, Loss: 313.0205
Epoch: 05 | Time: 0m 7s
	Train Loss: 306.458 | Val. Loss: 212.396
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 6, Batch 0, Loss: 305.1356
Epoch: 06 | Time: 0m 7s
	Train Loss: 297.214 | Val. Loss: 205.636
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 7, Batch 0, Loss: 289.2971
Epoch: 07 | Time: 0m 7s
	Train Loss: 288.701 | Val. Loss: 198.955
	-> 검증 손실 감소, 모델 저장 완료.
Epoch 8, Batch 0, Loss: 278.5552
Epoch: 08 | Time: 0m 7s
	Train Loss: 280.838 | Val. Loss: 192.011
	-> 검증 손

In [63]:
# 현재 상태 저장
torch.save({
    'model_state_dict': transformer.state_dict(),
    'train_loss': 173.287,
    'val_loss': 112.937
}, 'transformer_final_loss112.pt')

# 예측 테스트 실행
full_evaluation_report()

# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.
---
# 제출
Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>
Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2
Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
슝=3


In [64]:
# 현재 상태 저장
torch.save({
    'model_state_dict': transformer.state_dict(),
    'train_loss': 173.287,
    'val_loss': 112.937
}, 'transformer_final_loss112.pt')

# 예측 테스트 실행
full_evaluation_report()

# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.
---
# 제출
Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>
Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2
Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
슝=3


In [66]:
def predict_single(model, tokenizer, question):
   model.eval()
   
   # 토큰화 및 패딩
   q_tokens = tokenizer.encode_as_ids(question)[:50]
   q_tokens += [0] * (50 - len(q_tokens))
   
   src = torch.tensor([q_tokens]).to(device)
   tgt = torch.tensor([[1]]).to(device)  # 시작 토큰
   
   with torch.no_grad():
       for _ in range(30):
           enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)
           output = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
           
           if isinstance(output, tuple):
               output = output[0]
           
           next_token = output[0, -1].argmax().item()
           if next_token == 2:  # 종료 토큰
               break
           tgt = torch.cat([tgt, torch.tensor([[next_token]]).to(device)], dim=1)
   
   return tokenizer.decode_ids(tgt[0].tolist()[1:])

# 랜덤 테스트
random_indices = random.sample(range(len(questions)), 5)

for i, idx in enumerate(random_indices, 1):
   q = questions[idx]
   a = answers[idx]
   pred = predict_single(transformer, tokenizer, q)
   
   print(f"{i}. 질문: {q}")
   print(f"   정답: {a}")
   print(f"   예측: {pred}")
   print("-" * 30)

1. 질문: 집나가도 갈 데가 없어
   정답: 선생님이나 기관에 연락해보세요.
   예측: 나올까 무슨방비네요.
------------------------------
2. 질문: 비가 오니 이 노래가 생각 나네
   정답: 비와 당신.
   예측: 나올까를 울적하.
------------------------------
3. 질문: 썸 타고 있는데 주변에 말해?
   정답: 굳이 말할 필요는 없는 거 같아요.
   예측: 단순하을 마음은이라니요.
------------------------------
4. 질문: 섬유유연제 향 좋은거 사야겠지
   정답: 향이 많은 걸 말해주죠.
   예측: 싫어네요.
------------------------------
5. 질문: 6개월 가량의 이별과 재회의 끝
   정답: 아쉬운 이야기네요.
   예측: 용 볼 합니다.
------------------------------


In [67]:
import random

def test_random_questions(model, tokenizer, num_samples=5):
    """랜덤 질문으로 챗봇 테스트"""
    
    # 훈련 데이터에서 랜덤 질문 선택
    random_indices = random.sample(range(len(questions)), num_samples)
    
    print("# 랜덤 질문 테스트")
    print("=" * 50)
    
    model.eval()
    
    for i, idx in enumerate(random_indices, 1):
        original_question = questions[idx]
        original_answer = answers[idx]
        
        print(f"\n{i}. 질문: {original_question}")
        print(f"   정답: {original_answer}")
        
        try:
            # 모델 예측
            predicted_answer = predict_single(model, tokenizer, original_question)
            print(f"   예측: {predicted_answer}")
        except Exception as e:
            print(f"   예측: [실패: {str(e)}]")
        
        print("-" * 30)

# 실행
test_random_questions(transformer, tokenizer, num_samples=7)
print("슝=3")

# 랜덤 질문 테스트

1. 질문: 나 둘다 좋아하는 것 같애
   정답: 마음의 방이 두 개 있나봐요.
   예측: 명절그램 거예요.
------------------------------

2. 질문: 지금 가장 힘든 거는
   정답: 자신일 것입니다.
   예측: 또 다시 결혼하기은이라니요.
------------------------------

3. 질문: 동거하다가 헤어질까봐 걱정이야
   정답: 걱정하면 아무것도 못해요
   예측: 나올까인지밖요.
------------------------------

4. 질문: 먹을 게 일도 없어
   정답: 장 보러 갈 타이밍이네요.
   예측: 다가네요.
------------------------------

5. 질문: 뉴스는 역시 지루해
   정답: 흥미를 가져보세요.
   예측: 단순하을 마음은 괜찮아 맞은세요.
------------------------------

6. 질문: 무슨 매주 결혼식이야
   정답: 인맥이 넓으신가봐요.
   예측: 저도넌도름죠.
------------------------------

7. 질문: 집 아예 샀어
   정답: 내 집 마련 축하드려요.
   예측: 한났네고 있어요.
------------------------------
슝=3


In [68]:
# 원본 데이터만 사용 (증강 제거)
print("원본 데이터로 학습 준비")
print(f"원본 데이터: {len(questions)}개")

# 원본 데이터 토큰화
original_que_corpus = []
original_ans_corpus = []

for q, a in zip(questions, answers):
    q_tokens = tokenizer.encode_as_ids(q)
    a_tokens = [1] + tokenizer.encode_as_ids(a) + [2]  # 시작/끝 토큰 추가
    
    if len(q_tokens) <= 40 and len(a_tokens) <= 40:
        original_que_corpus.append(q_tokens)
        original_ans_corpus.append(a_tokens)

print(f"토큰화 완료: {len(original_que_corpus)}개")

# 패딩
original_enc_ndarray = pad_sequences_custom(original_que_corpus, max_len=MAX_LEN, pad_value=0)
original_dec_ndarray = pad_sequences_custom(original_ans_corpus, max_len=MAX_LEN, pad_value=0)

# 새 데이터로더
original_dataset = TensorDataset(original_enc_ndarray, original_dec_ndarray)
original_dataloader = DataLoader(original_dataset, batch_size=128, shuffle=True)

print(f"데이터셋 크기: {original_enc_ndarray.shape}")

# 새 모델로 학습
transformer_original = Transformer(
    n_layers=3, d_model=256, n_heads=8, d_ff=512,
    src_vocab_size=VOCAB_SIZE, tgt_vocab_size=VOCAB_SIZE,
    pos_len=200, dropout=0.2, shared_fc=True, shared_emb=True
).to(device)

optimizer_original = torch.optim.Adam(transformer_original.parameters(), lr=0.0001)

# 학습 시작
print("\n원본 데이터 학습 시작")
for epoch in range(1, 21):
    train_loss = train_epoch(transformer_original, original_dataloader, optimizer_original, criterion, device, epoch)
    print(f'Epoch: {epoch:02} | Train Loss: {train_loss:.3f}')

print("학습 완료!")

원본 데이터로 학습 준비
원본 데이터: 11823개
토큰화 완료: 11823개
데이터셋 크기: torch.Size([11823, 50])

원본 데이터 학습 시작
Epoch 1, Batch 0, Loss: 3270.3552
Epoch: 01 | Train Loss: 2383.629
Epoch 2, Batch 0, Loss: 1927.5568
Epoch: 02 | Train Loss: 1594.460
Epoch 3, Batch 0, Loss: 1366.5895
Epoch: 03 | Train Loss: 1316.358
Epoch 4, Batch 0, Loss: 1215.2081
Epoch: 04 | Train Loss: 1210.763
Epoch 5, Batch 0, Loss: 1140.7401
Epoch: 05 | Train Loss: 1139.874
Epoch 6, Batch 0, Loss: 1136.1343
Epoch: 06 | Train Loss: 1084.573
Epoch 7, Batch 0, Loss: 1118.2805
Epoch: 07 | Train Loss: 1037.138
Epoch 8, Batch 0, Loss: 1046.7931
Epoch: 08 | Train Loss: 993.556
Epoch 9, Batch 0, Loss: 994.2520
Epoch: 09 | Train Loss: 954.346
Epoch 10, Batch 0, Loss: 943.1646
Epoch: 10 | Train Loss: 917.952
Epoch 11, Batch 0, Loss: 924.1323
Epoch: 11 | Train Loss: 881.817
Epoch 12, Batch 0, Loss: 891.4734
Epoch: 12 | Train Loss: 850.255
Epoch 13, Batch 0, Loss: 840.0701
Epoch: 13 | Train Loss: 819.386
Epoch 14, Batch 0, Loss: 846.3079
Epoch: 14 |

In [69]:
# 두 모델 성능 비교
print("=== 모델 성능 비교 ===")
random_idx = random.randint(0, len(questions)-1)
test_q = questions[random_idx]
test_a = answers[random_idx]

print(f"질문: {test_q}")
print(f"정답: {test_a}")

# 증강 모델 예측
pred1 = predict_single(transformer, tokenizer, test_q)
print(f"증강 모델: {pred1}")

# 원본 모델 예측  
pred2 = predict_single(transformer_original, tokenizer, test_q)
print(f"원본 모델: {pred2}")

=== 모델 성능 비교 ===
질문: 딱 알아봤는데
정답: 여전하던가요?
증강 모델: 랜선 구썸남이 귀찮은.
원본 모델: 


In [70]:
# 두 모델 성능 비교
print("=== 모델 성능 비교 ===")
random_idx = random.randint(0, len(questions)-1)
test_q = questions[random_idx]
test_a = answers[random_idx]

print(f"질문: {test_q}")
print(f"정답: {test_a}")

# 증강 모델 예측
pred1 = predict_single(transformer, tokenizer, test_q)
print(f"증강 모델: {pred1}")

# 원본 모델 예측  
pred2 = predict_single(transformer_original, tokenizer, test_q)
print(f"원본 모델: {pred2}")

=== 모델 성능 비교 ===
질문: 할 줄 아는거!
정답: 많은걸 하고 싶은데 아직 못하는게 많아요.
증강 모델: 것도 중요한도 진심으로 오래 해보세요.
원본 모델: 


In [71]:
# 두 모델 성능 비교
print("=== 모델 성능 비교 ===")
random_idx = random.randint(0, len(questions)-1)
test_q = questions[random_idx]
test_a = answers[random_idx]

print(f"질문: {test_q}")
print(f"정답: {test_a}")

# 증강 모델 예측
pred1 = predict_single(transformer, tokenizer, test_q)
print(f"증강 모델: {pred1}")

# 원본 모델 예측  
pred2 = predict_single(transformer_original, tokenizer, test_q)
print(f"원본 모델: {pred2}")

=== 모델 성능 비교 ===
질문: 나를 좋아하는 사람을 만나는 건 축복일거야.
정답: 서로 사랑하는 건 기적이죠.
증강 모델: 단순하을 마음은이라니가 누나.
원본 모델: 


In [72]:
# 1. 원본 데이터만으로 더 오래 학습
EPOCHS = 50
for epoch in range(21, EPOCHS + 1):
    train_loss = train_epoch(transformer_original, original_dataloader, optimizer_original, criterion, device, epoch)
    print(f'Epoch: {epoch:02} | Train Loss: {train_loss:.3f}')

# 2. 또는 학습률 조정
optimizer_original = torch.optim.Adam(transformer_original.parameters(), lr=0.0005)

Epoch 21, Batch 0, Loss: 611.7399
Epoch: 21 | Train Loss: 632.132
Epoch 22, Batch 0, Loss: 599.3767
Epoch: 22 | Train Loss: 614.755
Epoch 23, Batch 0, Loss: 611.4449
Epoch: 23 | Train Loss: 593.914
Epoch 24, Batch 0, Loss: 585.0062
Epoch: 24 | Train Loss: 575.738
Epoch 25, Batch 0, Loss: 544.5309
Epoch: 25 | Train Loss: 558.846
Epoch 26, Batch 0, Loss: 530.2510
Epoch: 26 | Train Loss: 540.296
Epoch 27, Batch 0, Loss: 535.3284
Epoch: 27 | Train Loss: 523.848
Epoch 28, Batch 0, Loss: 533.0620
Epoch: 28 | Train Loss: 507.021
Epoch 29, Batch 0, Loss: 490.6982
Epoch: 29 | Train Loss: 489.746
Epoch 30, Batch 0, Loss: 489.9977
Epoch: 30 | Train Loss: 474.935
Epoch 31, Batch 0, Loss: 445.6504
Epoch: 31 | Train Loss: 458.030
Epoch 32, Batch 0, Loss: 481.8080
Epoch: 32 | Train Loss: 442.879
Epoch 33, Batch 0, Loss: 433.1244
Epoch: 33 | Train Loss: 428.662
Epoch 34, Batch 0, Loss: 402.5423
Epoch: 34 | Train Loss: 414.925
Epoch 35, Batch 0, Loss: 421.4415
Epoch: 35 | Train Loss: 400.803
Epoch 36, 

In [74]:
# 몇 개 질문으로 응답 품질 확인
for i in range(3):
    idx = random.randint(0, len(questions)-1)
    q = questions[idx]
    a = answers[idx]
    pred = predict_single(transformer_original, tokenizer, q)
    
    print(f"질문: {q}")
    print(f"정답: {a}")
    print(f"예측: {pred}")
    print("-" * 30)

질문: 의욕이 없어
정답: 가벼운 산책을 해보세요.
예측: 
------------------------------
질문: 한잔 했습니다
정답: 한 잔 하기 좋은 날이네요.
예측: 
------------------------------
질문: 연락하지 말라고 했어
정답: 때로는 단호한 것도 필요해요.
예측: 
------------------------------


In [75]:
# 예측 과정 디버깅
def debug_predict(model, tokenizer, question):
    model.eval()
    q_tokens = tokenizer.encode_as_ids(question)[:50]
    q_tokens += [0] * (50 - len(q_tokens))
    
    src = torch.tensor([q_tokens]).to(device)
    tgt = torch.tensor([[1]]).to(device)  # 시작 토큰
    
    print(f"입력 토큰: {q_tokens[:10]}...")
    print(f"시작 토큰 ID: 1")
    
    with torch.no_grad():
        for step in range(5):  # 처음 5스텝만 확인
            enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)
            output = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
            
            if isinstance(output, tuple):
                output = output[0]
            
            next_token = output[0, -1].argmax().item()
            print(f"스텝 {step}: 예측 토큰 {next_token}")
            
            if next_token == 2:  # 종료 토큰
                print("종료 토큰 생성됨")
                break
            tgt = torch.cat([tgt, torch.tensor([[next_token]]).to(device)], dim=1)
    
    return tokenizer.decode_ids(tgt[0].tolist()[1:])

# 테스트
test_q = "안녕하세요"
result = debug_predict(transformer_original, tokenizer, test_q)
print(f"최종 결과: {result}")

입력 토큰: [1, 3160, 2, 0, 0, 0, 0, 0, 0, 0]...
시작 토큰 ID: 1
스텝 0: 예측 토큰 1
스텝 1: 예측 토큰 1
스텝 2: 예측 토큰 1
스텝 3: 예측 토큰 1
스텝 4: 예측 토큰 1
최종 결과: 


In [77]:
# 1. 11에포크 이전 모델로 복원 (만약 저장했다면)
# transformer_new.load_state_dict(torch.load('epoch_10_model.pt'))

# 2. 학습률 대폭 감소
optimizer_stable = torch.optim.Adam(transformer_new.parameters(), lr=0.0001)  # 원래대로

# 3. 안정적인 학습 재시작
print("안정적인 학습률로 재시작")
for epoch in range(1, 31):
    train_loss = train_epoch(transformer_new, small_dataloader, optimizer_stable, criterion, device, epoch)
    print(f'Epoch: {epoch:02} | Train Loss: {train_loss:.3f}')
    
    # 손실 급등 방지
    if epoch > 1 and train_loss > 500:
        print("손실 급등 감지, 학습률 추가 감소")
        for param_group in optimizer_stable.param_groups:
            param_group['lr'] *= 0.5

안정적인 학습률로 재시작
Epoch 1, Batch 0, Loss: 1276.8003
Epoch 1, Batch 100, Loss: 895.1368
Epoch 1, Batch 200, Loss: 893.0211
Epoch 1, Batch 300, Loss: 734.6543
Epoch: 01 | Train Loss: 897.526
Epoch 2, Batch 0, Loss: 881.9675
Epoch 2, Batch 100, Loss: 926.4646
Epoch 2, Batch 200, Loss: 807.7474
Epoch 2, Batch 300, Loss: 756.7810
Epoch: 02 | Train Loss: 781.712
손실 급등 감지, 학습률 추가 감소
Epoch 3, Batch 0, Loss: 758.0356
Epoch 3, Batch 100, Loss: 732.5965
Epoch 3, Batch 200, Loss: 642.7484
Epoch 3, Batch 300, Loss: 701.4048
Epoch: 03 | Train Loss: 713.104
손실 급등 감지, 학습률 추가 감소
Epoch 4, Batch 0, Loss: 634.2913
Epoch 4, Batch 100, Loss: 676.2718
Epoch 4, Batch 200, Loss: 709.1272
Epoch 4, Batch 300, Loss: 615.0367
Epoch: 04 | Train Loss: 682.388
손실 급등 감지, 학습률 추가 감소
Epoch 5, Batch 0, Loss: 734.1345
Epoch 5, Batch 100, Loss: 775.0132
Epoch 5, Batch 200, Loss: 661.0947
Epoch 5, Batch 300, Loss: 692.5763
Epoch: 05 | Train Loss: 667.813
손실 급등 감지, 학습률 추가 감소
Epoch 6, Batch 0, Loss: 713.5786
Epoch 6, Batch 100, Lo

## 회고

우선 이번프로젝트 내내 감기기운 + 약기운 때문에 집중도 낮고 그래서 진도따라가기도 어려웠고 프로젝트를 번역노드쪽을 진행하다가 잘못진행한걸 알게되어서 프로젝트 자체도 늦게 시작.. 해결하지못하는 오류 때문에 계속 버벅임
### RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

최대한 해보려고 했으나,, 데이터 증강의 퀄리티도 낮고, 왜낮은지도 모르겠고,, ko파일 받아서 했는데 데이터 증강 품질이 너무 안좋아서 챗봇 대답이 이상한 외계어가 나옴.. 그렇다고 원본 데이터로 하려고하니 학습이 잘 되지않고 챗봇 대답 또한 낮음,, 

전반적으로 몸 컨디션 관리에 실패해서 팀원분들과 속도 맞추는데도 실패했음.. 그래도 포기하지않고 약의 힘으로 버텨보지만 내일,, 병원 갓다와서 다른분들 하신거 보면서 다시 공부해봐야겠다........ 그래도 어느정도 트랜스포머 모델 구조나 코드, 실험하는 방법 설계 등등 조금씩은 배워가는 것 같다. 포기하지않고,, 계속해보겠습니다..